This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 10000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'what_offset_std': 15, 'what_offset_max': 25, 'N_pic': 128, 'offset_std': 30, 'offset_max': 34, 'noise': 0.75, 'contrast': 0.7, 'sf_0': 0.1, 'B_sf': 0.1, 'N_theta': 6, 'N_azimuth': 24, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.0, 'dim1': 1000, 'dim2': 1000, 'lr': 0.005, 'do_adam': True, 'bn1_bn_momentum': 0.5, 'bn2_bn_momentum': 0.5, 'momentum': 0.3, 'epochs': 60, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2020-02-02', 'seed': 2019, 'N_cv': 10, 'do_compute': True, 'save_model': True}
Overwriting train.py
2020-02-18T08:43:42+01:00

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
matplotlib 3.1.3
torch 1.4.0


In [2]:
figname = '../figures/fig_params'

In [3]:
%%writefile params.py
import os
import torch
from main import init, MetaML
from where import Where as ML
from what import WhatNet
import numpy as np
import sys
opts = dict(filename='../data/2020-02-02', verbose=0, log_interval=0, do_compute=False  if len(sys.argv) > 1 else True)
#opts = dict(filename='debug', verbose=0, log_interval=0)
print('do_compute =', opts['do_compute'])

print(50*'-')
print(' parameter scan')
print(50*'-')

if opts['do_compute']:
    args = init(**opts)
    #args.filename = '../data/2019-03-27'
    filename_train = args.filename + '_train.pt'
    if not(os.path.isfile(filename_train + '_lock')):
        open(filename_train + '_lock', 'w').close()
        print(50*'-')
        print('Default parameters')
        print(50*'-')
        from where import Where as ML
        from what import WhatNet
        ml = ML(args)
        ml.train(path=filename_train)
        # ml.main(path=args.filename)
        try:
            os.remove(filename_train + '_lock')
        except:
            pass

if True:
    args = init(**opts)
    mml = MetaML(args)
    if torch.cuda.is_available():
        mml.scan('no_cuda', [True, False])

    args = init(**opts)
    mml = MetaML(args)
    mml.scan('bias_deconv', [True, False])

results = {}
    
def update_results(results, parameter, accuracies, ci=0.01):
    from scipy.stats import beta

    if not parameter in results.keys(): results[parameter] = dict(value=[], accuracy=[], p_low=[], p_sup=[])
    for value in accuracies.keys():
        results[parameter]['value'].append(value)
        results[parameter]['accuracy'].append(accuracies[value][:-1].mean()*100)
        try:
            a1, b1, loc1, scale1 = beta.fit(accuracies[value][:-1], floc=0, fscale=1)
            p_low, p_sup = beta.ppf([ci, 1-ci], a=a1, b=b1)
            #print(p_low, p_sup)
            results[parameter]['p_low'].append(p_low*100)
            results[parameter]['p_sup'].append(p_sup*100)
        except:
            results[parameter]['p_low'].append(accuracies[value][:-1].mean()*100)
            results[parameter]['p_sup'].append(accuracies[value][:-1].mean()*100)
        
    return results
            
bases = [np.sqrt(2), 2, 2*np.sqrt(2)]
bases = [2]
bases = [np.sqrt(2), 2]

for base in bases if not args.filename == '../data/debug' else [2]:
    print(50*'-')
    print(' base=', base)
    print(50*'-')

    print(50*'-')
    print(' parameter scan : data')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args, base=base)
    for parameter in ['sf_0', 'B_sf', 'offset_std' , 'noise', 'contrast']: #
        accuracies = mml.parameter_scan(parameter)
        results = update_results(results, parameter, accuracies)
        
    print(50*'-')
    print(' parameter scan : network')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['dim1',
                      'bn1_bn_momentum',
                      'dim2',
                      'bn2_bn_momentum',
                      'p_dropout']:
        accuracies = mml.parameter_scan(parameter)
        results = update_results(results, parameter, accuracies)

    print(' parameter scan : learning ')
    args = init(**opts)
    mml = MetaML(args, base=base, tag='SGD')
    print(50*'-')
    print('Using SGD')
    print(50*'-')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        accuracies = mml.parameter_scan(parameter)
        results = update_results(results, parameter + '_sgd', accuracies)
    print(50*'-')
    print('Using ADAM')
    print(50*'-')
    args = init(**opts)
    args.do_adam = True
    mml = MetaML(args, base=base, tag='adam')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        if not (base == 2 and parameter=='epochs'): # HACK
            accuracies = mml.parameter_scan(parameter)
            results = update_results(results, parameter + '_adam', accuracies)

    print(50*'-')
    print(' parameter scan : retina')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['N_theta',
                      'N_azimuth',
                      'N_eccentricity',
                      'rho']:
        accuracies = mml.parameter_scan(parameter)
        results = update_results(results, parameter, accuracies)
        

Overwriting params.py


In [4]:
%run params.py scan

do_compute = False
--------------------------------------------------
 parameter scan
--------------------------------------------------
scanning over bias_deconv = [True, False]
For parameter bias_deconv = True ,  loading model


/usr/local/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'where.WhereNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Loading training dataset
Loading testing dataset


NameError: name 'args' is not defined

## parameter scan

In [5]:
def plot_accuracy(result, ymin=0, ychance=10, fig=None, ax=None, xlabel=None):
    if fig is None: fig, ax = plt.subplots(figsize=(5, 5))
    #ax.plot(result['accuracy'], result['value'], "o", color='blue')
    # print(value, accuracy)
    ax.hlines(y=ychance, xmin=np.min(result['value']), xmax=np.max(result['value']), color='black', linestyles='dashed')
    ax.vlines(x=result['value'], ymin=result['accuracy'], ymax=100, color='red')
    ax.vlines(x=result['value'], ymin=ymin, ymax=result['accuracy'], color='skyblue')
    ax.vlines(x=result['value'], ymin=result['p_low'], ymax=result['p_sup'], color='blue', alpha=1., lw=7)

    if xlabel is None:
        xlabel = parameter.capitalize()
    ax.set_xlabel(xlabel)
    #print( results[parameter]['value'] )
    #print( np.min([np.float(k) for k in results[parameter]['value']]))
    #ax.vlines(100, np.min(results[parameter]['value']), np.max(results[parameter]['value']), 'k', label='max')
    ax.set_ylabel('Accuracy (%)')
    # Only draw spine between the y-ticks
    #ax.spines['left'].set_bounds(-1, 1)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    return fig, ax

In [6]:
figname_ = 'accuracy_'
figname_ = ''

tex = ''
for parameter in results.keys():
    print('--->',  parameter)
    fig, ax = plot_accuracy(results[parameter])
    plt.show()
    if not figname_ == '':
        fig_dir = './'
        fig_ext = '.pdf'
        fname = figname_ + parameter + fig_ext
        tex += "\includegraphics[width=.0666\linewidth]{"+ fname + "} \n"
        fig.savefig(os.path.join(fig_dir, fname),
                bbox_inches='tight', pad_inches=0)
print(tex)

NameError: name 'results' is not defined

# summary


In [ ]:
results.keys()

In [ ]:
parameters = [['noise', 'offset_std', 'sf_0', 'B_sf'], ['epochs_adam', 'dim1', 'dim2', 'N_azimuth']]  # '', 'lr', 
fig, axs = plt.subplots(len(parameters), len(parameters[0]), figsize = (figwidth, figwidth/len(parameters)*1.1), subplot_kw={})

for i_row, parameters_ in enumerate(parameters):
    for ax, parameter in zip(axs[i_row], parameters_):
        fig, ax = plot_accuracy(results[parameter], fig=fig, ax=ax)

for ax, text, x_offset, y_offset in [[axs[1][0], 'B', -.35, .95], [axs[1][-1], 'C', -.35, .95], [axs[0][0], 'A', -.35, .95]]:
    ax.text(x_offset, y_offset, '(' + text + ')', fontsize=24,
              bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
              ha='left', va='center', transform=ax.transAxes) 
        
plt.tight_layout()
fig.savefig(figname + '.pdf', bbox_inches='tight', pad_inches=0.05)

# post-processing

In [ ]:
!convert  -density {dpi_export} {figname}.pdf {figname}.jpg
!convert  -density {dpi_export} {figname}.pdf {figname}.png
#!convert  -density {dpi_export} -resize 5400  -units pixelsperinch -flatten  -compress lzw  -depth 8 {fname}.pdf {fname}.tiff

In [ ]:
from IPython.display import Image
Image('{figname}.png'.format(figname=figname))

In [ ]:
!ls  -l {figname}*